# Residual connection

The residual block was introduced by *Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun* in the paper *Deep Residual Learning for Image Recognition*.

## Theory


### a) 

Draw a simple residual block and mention/**find** som of the key advantages of including residual connections.



**Solution:**

<img src="basic_block.jpg" alt="drawing" width="200"/>

Residual connections introduces building blocks on the form $f(x) = F(x, W) + x$. This improves training in many aspects:

* It allows training deep layer networks, as unneccessary layers could in theory have their weights go to zero, only passing on the identity to the next layer. Thus larger networks are a superset of smaller networks, and if the smaller network was more optimal, the weights of the additional layers in the large network would ideally go to zero. Thus the finally trained networks would simply skip those redundant layers.
* In backpropagation the resiudal connections provide sort of a *highway* for gradients, fighting the vanishing gradient problem. $f'(x) = \frac{\delta}{\delta x} F(x,W) + 1$

### b)

The plain residual block performs an identity mapping from input to output. **Given** a downsampler block (a block where the dimensions are reduced by strided convolutions) that downsamples from $38x38x256$ to $14x14x512$ shape. **Find** how one can apply residual connection in this case.

**Solution:**

It is clear that a simple identity doesn't suffice in this example, as the input and output shape differ.

There are to different aspects here to address. One is that the height and width is reduced. The solution to this is to take the identity with a stride of two, instead of taking every pixel.

The second is that the depth is increased. The Resnet paper proposes two solutions to this: zero-pad the missing channels, or use 1x1-convolutions. The 1x1 convolution turns out to be most successfull.

Final solution: Change the block function to $f(x) = F(x, W) + h(x, w)$ where $h(x,w)$ is a 1x1 convolution block with input = 256, output = 512, stride = 2, pad = 0. 

## Numerical example

### Bottleneck block

This image shows a bottle neck block.

<img src="bottleneck.jpg" alt="drawing" width="200"/>

**Given** an input feature map of 38x38x256, compare the bottle neck block to a standard basic block consisting of two 3x3 convolutions in terms of parameters and FLOP. The convolution operations do not apply bias. In other words **find** parameters and FLOPs for both a basic block and a bottleneck block with an input and output feature map of 38x38x256. Give a short comment on the result.


**Solution:**

**Basic block:** 
* \# Parameters:
    * Each conv layer has $3*3*256*256$
    * Resulting in 2 x \# per conv layer equals <ins>1 179 648</ins> parametes
* \# Floating point operations:
    * Each output pixel is the result of a 3x3x256 convolution operation, giving 9x256 multiplications. A summation of 2304 values, gives 2303 distinct summations.
    * The total for one layer is then 256 output feature, multiplied with 38x38 pixels and 4607 operations per pixel. Giving 1 703 042 048 operations.
    * Addition with the identity after the convolution operations is 38x38x256 summations
    * There are two layers, and addition with the identity at the end which gives <ins>3 406 453 760</ins> operations.
    
**Bottle neck:**
* \# Parameters:
   * The first conv layer has $1x1x256x64$ parameters, the second conv layer has $3x3x64x64$ parameters and the third layer has $1x1x64x256$ parameters.
   * Total number of parameters are then <ins>69 632</ins> parameters.

* \# Floating point operations:
    * Each output pixel in the 1x1x64 convolution operation is the result of 256 multiplications and 255 additions giving 511 operations per pixel. With 38x38x64 pixels in the output feature map there are 47 224 576 operations in this layer.
    * Each output pixel in the 3x3x64 convolution operation is the result of $3*3*64$ multiplications and $575$ additions giving 1151 operations per pixel. With 38x38x64 output pixels there are 106 370 816 operations in this layer.
    * Each output pixel in the 1x1x256 convolution operation is the result of 64 multiplications and 63 additions, giving 127 operations per pixel. With 38x38x256 pixels in the output feature map there are 46 947 328 operations in this layer.
    * Addition with the identity still gives 38x38x256 summations
    * Total number of operations are then <ins>200 912 384</ins>.

Altough the calculations may be inaccurate as to what operations would actually be performed in the computer, as the GPU in reality would perform MAC operations or similar instead of separate multiplications and additions, this gives a clear picture that the bottle neck blocks reduces both the time and space complexity of the model. Reducing the complexity of each block in turn allows for deeper networks, as deep networks of basic blocks would be too computationally expensive.
    